In [ ]:
!pip install alerce
import pandas as pd
from alerce.core import Alerce
from astropy.time import Time
import numpy as np
import matplotlib.pylab as plt
import gzip
import warnings
from PIL import Image
import numpy as np
import random
import os

# Suppress all warnings
warnings.filterwarnings("ignore")

client = Alerce()

def get_objects_per_class(classearly="SN", pclassearly=0.6, min_nobs=10, max_nobs=50, n_objects=100):

    min_firstmjd = Time("2017-11-01T00:00:00", format="isot", scale="utc").mjd + 70

    objects = client.query_objects(classifier="stamp_classifier",
                                   class_name=classearly,
                                   probability=pclassearly,
                                   ndet=[1, 50],
                                   #order_by="probability",
                                   #order_mode="DESC",
                                   count=False,
                                   first_mjd=[min_firstmjd, None],
                                   page_size=n_objects,
                                   format='pandas')
    objects.head()
    objects.set_index("oid", inplace=True)
    objects.sort_values(by="ndet", inplace=True, ascending=False)
    return objects

n_objects = 10 # Objects per class to query
early_classes = ["AGN", "SN", "VS", "asteroid", "bogus"] # Class identifiers to query objects
objects = {} # Initialize dictionary to use the results per class

min_nobs = 1
difference_images = {}

for i, cl in enumerate(early_classes):
    print(f"Class: {cl}")
    objects[cl] = get_objects_per_class(classearly=cl, min_nobs=min_nobs, n_objects=n_objects)
    if i == 0:
        print("Result of a query using the ALeRCE client")
        display(objects[cl].head())
        print("Columns available:", objects[cl].columns)

    # Loop through all objects in this class
    for oid in objects[cl].index:
        try:
          stamps = client.get_stamps(oid)
        except gzip.BadGzipFile:
          print(f"Skipping")
        if np.isnan(stamps[2].data).any():
          pass
        else:
          difference_images[cl + "_" + str(stamps[2])] = stamps[2].data

# print(list(difference_images.keys()), list(difference_images.values()))

class_encoding = {"AGN" : 0, "SN" : 1, "VS": 2, "asteroid" : 3, "bogus": 4}

train_array = []
test_array = []
val_array = []


def stamps_to_image(dicti):
  keys = list(dicti.keys())  # Convert to list for shuffling
  random.shuffle(keys)  # Shuffle the list of keys
  image = 0
  num_images = len(keys)

  train_keys = keys[:int(num_images * 0.6)]
  val_keys = keys[int(num_images * 0.6):int(num_images * 0.7)]
  test_keys = keys[int(num_images * 0.7):]

  for key in train_keys:

    parts = key.split("_")
    # The first part should be the 'cl' value
    cl = parts[0]

    # save_dir = cl
    os.makedirs("train", exist_ok=True)
    # Convert the image data to a Pillow Image
    img_data = dicti[key]

    # Normalize the image data to 8-bit integer format (0-255)
    img_data = np.nan_to_num(img_data, nan=0.0, posinf=255, neginf=0)
    img_data_normalized = ((img_data - np.min(img_data)) / (np.max(img_data) - np.min(img_data)) * 255).astype(np.uint8)

    # Convert the normalized data to a Pillow Image
    img = Image.fromarray(img_data_normalized)

    # Save each stamp as a PNG file
    train_array.append(class_encoding[cl])
    img.save(f"train/{str(image).zfill(4)}.jpg")
    image = image + 1
    print(image, "train")


  image = 0
  for key in val_keys:
    parts = key.split("_")
    # The first part should be the 'cl' value
    cl = parts[0]

    # save_dir = cl
    os.makedirs("val", exist_ok=True)
    # Convert the image data to a Pillow Image
    img_data = dicti[key]

    # Normalize the image data to 8-bit integer format (0-255)
    img_data = np.nan_to_num(img_data, nan=0.0, posinf=255, neginf=0)
    img_data_normalized = ((img_data - np.min(img_data)) / (np.max(img_data) - np.min(img_data)) * 255).astype(np.uint8)

    # Convert the normalized data to a Pillow Image
    img = Image.fromarray(img_data_normalized)

    # Save each stamp as a PNG file
    val_array.append(class_encoding[cl])
    img.save(f"val/{str(image).zfill(4)}.jpg")
    image = image + 1
    print(image, "val")


  image = 0


  for key in test_keys:
    parts = key.split("_")
    # The first part should be the 'cl' value
    cl = parts[0]

    # save_dir = cl
    os.makedirs("test", exist_ok=True)
    # Convert the image data to a Pillow Image
    img_data = dicti[key]

    # Normalize the image data to 8-bit integer format (0-255)
    img_data = np.nan_to_num(img_data, nan=0.0, posinf=255, neginf=0)
    img_data_normalized = ((img_data - np.min(img_data)) / (np.max(img_data) - np.min(img_data)) * 255).astype(np.uint8)

    # Convert the normalized data to a Pillow Image
    img = Image.fromarray(img_data_normalized)

    # Save each stamp as a PNG file
    test_array.append(class_encoding[cl])
    img.save(f"test/{str(image).zfill(4)}.jpg")
    image = image + 1
    print(image, "test")

stamps_to_image(difference_images)
print("length val array:", val_array)
print("len test array:", test_array)
print("len train array:", train_array)




Class: AGN
Result of a query using the ALeRCE client


,ndethist,ncovhist,mjdstarthist,mjdendhist,corrected,stellar,ndet,g_r_max,g_r_max_corr,g_r_mean,...,lastmjd,deltajd,meanra,meandec,sigmara,sigmadec,class,classifier,probability,step_id_corr
oid,,,,,,,,,,,,,,,,,,,,,
ZTF19aatvxff,106,1663,58582.500000,60123.500000,True,False,41,1.178200,1.862864,0.826642,...,60123.428519,1540.959965,328.686831,39.550045,0.000086,0.000048,AGN,stamp_classifier,0.767011,1.2.0
ZTF19aazppwn,186,1293,58636.500000,59813.250000,True,False,41,NaN,NaN,NaN,...,59813.257419,1176.825532,319.245075,16.109548,0.000047,0.000038,AGN,stamp_classifier,0.696602,dev
ZTF18achankm,75,1938,58302.452627,60531.340046,True,False,39,0.293831,0.532726,0.401135,...,60531.340046,2099.157465,327.055638,43.016960,0.088901,0.065000,AGN,stamp_classifier,0.670109,24.5.2a6
ZTF18abapzgs,118,3692,58283.381285,60534.302454,True,False,39,0.697533,1.197698,0.709319,...,60534.302454,2230.865359,318.642963,22.617149,0.065111,0.060104,AGN,stamp_classifier,0.717364,24.5.2a6
ZTF18acrhyxw,102,2128,58104.137211,60540.286979,True,False,38,0.019165,0.520605,0.228564,...,60540.286979,1906.860718,311.039515,34.680949,0.062789,0.051633,AGN,stamp_classifier,0.717124,24.5.2a6


Columns available: Index(['ndethist', 'ncovhist', 'mjdstarthist', 'mjdendhist', 'corrected',
       'stellar', 'ndet', 'g_r_max', 'g_r_max_corr', 'g_r_mean',
       'g_r_mean_corr', 'firstmjd', 'lastmjd', 'deltajd', 'meanra', 'meandec',
       'sigmara', 'sigmadec', 'class', 'classifier', 'probability',
       'step_id_corr'],
      dtype='object')


In [ ]:
 !pip install mido pillow numpy

from PIL import Image
import numpy as np
import mido
from mido import MidiFile, MidiTrack, Message

def image_to_midi(image_path, output_midi_path):
    # Open the image
    img = Image.open(image_path).convert('L')  # Convert to grayscale
    img_array = np.array(img)

    # Flatten the image array and normalize to MIDI note range (0-127)
    img_array = img_array.flatten()
    img_array = np.interp(img_array, (0, 255), (0, 127)).astype(int)

    # Create a MIDI file
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)

    # Add MIDI events
    for note in img_array:
        track.append(Message('note_on', note=note, velocity=64, time=0))
        track.append(Message('note_off', note=note, velocity=64, time=480))  # Duration of each note

    # Save the MIDI file
    mid.save(output_midi_path)

# Example usage
image_path = '/content/bogus/bogus_<astropy.io.fits.hdu.image.ImageHDU object at 0x7bd822f70520>.png'
output_midi_path = 'sound.mid'
image_to_midi(image_path, output_midi_path)
